<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
 <b>Attributzugriff und Deskriptoren</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_220_advanced_topics/topic_140_e4_descriptors</div>


 ## Für Experten: Zugriff auf Attribute

 Python ermöglicht es uns als Programmierer, an mehreren Stellen in den
 Zugriff auf Attribute einzugreifen und das Verhalten zu modifizieren.

In [ ]:
class FortyTwo:
    def __get__(self, instance, objtype=None):
        return 42

In [ ]:
class MyClass:
    question = "???"
    answer = FortyTwo()

    def answer_question(self):
        print(f"The answer to {self.question} is {self.answer}.")

In [ ]:
my_obj = MyClass()
my_obj.answer_question()

In [ ]:
my_obj.question = "the ultimate question of life, the universe and everything"
my_obj.answer_question()

In [ ]:
my_obj.answer = "unknown"
my_obj.answer_question()

In [ ]:
# noinspection PyRedeclaration
class FortyTwo:
    def __get__(self, instance, objtype=None):
        return 42

    def __set__(self, instance, value):
        print("Don't try to change the facts!")

In [ ]:
class MyClass:
    question = "???"
    answer = FortyTwo()

    def answer_question(self):
        print(f"The answer to {self.question} is {self.answer}.")

In [ ]:
my_obj = MyClass()
my_obj.answer_question()

In [ ]:
my_obj.question = "the ultimate question of life, the universe and everything"
my_obj.answer_question()

In [ ]:
my_obj.answer = "unknown"
my_obj.answer_question()


 ## Attribute von Klassen

 Beim Zugriff auf `C.name` verfährt Python folgendermaßen:

 - Falls `name` ein Key in `C.__dict__` ist:
   - `v = C.__dict__['name']`
   - Falls `v` ein Deskriptor ist (i.e., `type(v).__get__` definiert ist:
     - Resultat ist `type(v).__get__(v, None, C)`
   - Falls `v` kein Deskriptor ist:
     - Resultat ist `v`
 - Falls `name` kein Key in `C.__dict__` ist:
   - Die Baisklassen von `C` werden in Method Resolution Order durchlaufen und
     diese Verfahren wird für jede Klasse ausgeführt


 ## Attribute von Instanzen

 Beim Zugriff auf `object.name` verfährt Python folgendermaßen:

 - Falls `name` ein Overriding Descriptor `v` in `C` oder einer der
   Basisklassen von `C` ist (`type(v)` hat Methoden `__get__()` und
   `__set__()`):
   - Das Resultat ist `type(v).__get__(v, object, C)`
 - Andernfalls, falls `name` ein Schlüssel in `object.__dict__` ist:
   - Das Resultat ist `object.__dict__['name']`
 - Andernfalls delegiert `object.name` die Suche an die Klasse, wie oben
   beschrieben
   - Falls dadurch ein Deskriptor `v` gefunden wird, so ist das Ergebnis
     `type(v).__get__(v, object, C)`
   - Wenn ein Wert `v` gefunden wird, der kein Deskriptor ist, dann wird `v`
     zurückgegeben
 - Wenn kein Wert gefunden wird und `C.__getattr__` definiert ist, dann wird
   `C.__getattr__(object, 'name')` aufgerufen um den Wert zu erhalten
 - Andernfalls wird eine `AttributeError` Exception ausgelöst

 Dieser Prozess kann durch die `__getattribute__` Methode überschrieben werden.

In [ ]:
class LoggingDescriptor:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print(f"__get__({self}, {instance}, {owner})")
        print(f"  __dict__ == {instance.__dict__}")
        return instance.__dict__.get(self.name, "nothing")

In [ ]:
class OverridingLoggingDescriptor(LoggingDescriptor):
    def __set__(self, instance, value):
        print(f"__set__({self}, {instance}, {value}")
        instance.__dict__[self.name] = value

In [ ]:
class YourClass:
    f = LoggingDescriptor("f")
    g = OverridingLoggingDescriptor("g")

In [ ]:
yc = YourClass()
print(yc.f, yc.g)

In [ ]:
yc.f = 234
yc.g = 345

In [ ]:
print(yc.f, yc.g)

In [ ]:
class MyClass:
    def g(self, x):
        print(self, x)


def f(x, y):
    print(x, y)

In [ ]:
mc = MyClass()
print(mc.__class__)

In [ ]:
print(MyClass.g)
print(mc.g.__qualname__)
print(mc.g.__get__)

In [ ]:
print(f.__get__)

In [ ]:
bound_f = f.__get__(mc, MyClass)
bound_g = mc.g
print(bound_f)
print(bound_g)

In [ ]:
bound_f(3)
bound_g(3)
mc.g(3)

In [ ]:
MyClass.f = f
mc.f(3)
mc.g(3)